In [1]:
#imports

%matplotlib inline
import pandas as pd
import numpy as np
import time
import requests
import bs4
import json
from flatten_json import flatten

In [64]:
# 20 requests every 1 seconds(s)
# 100 requests every 2 minutes(s)

apikey = "RGAPI-6f86b22f-3c4c-4f21-a277-3e191208dd48"

In [7]:
# get challenger summonerId from league-exp-api
# 1.2 delay bc short list

req = requests.get("https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/MASTER/I?page=1&api_key={}"
                   .format(apikey),time.sleep(1.2))
assert req.status_code == 200
data = req.json()
sumids = [d['summonerId'] for d in data]

In [8]:
sumids[0:5]

['8ELhquZ9m-r9QjqBMNGIkFr4frNobvMK1mijCMThwQwG8rwj',
 'XoeBmH1CDhS_vZSPlZ2ZfeW07nctWZrf1Ola7jgH2KgAHaE',
 'BiSrYLpEjcYJZFt70P74PzBV9rmCT27GV6X_nXWUC1vhPDs',
 'dqLpCtowSbSW0Q20P5DwlVT23zboeaCc851qND6BWCnGEh4',
 'kvy90UreuVrmRsREbYKmL19s2UDlTrtNJo6m6AyAM0sXd9g']

In [9]:
# get accountId via summonerId via summoner-api
# 1.2 delay bc short list

accids = []
for sumid in sumids:
    sumreq = requests.get("https://euw1.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}"
                          .format(sumid,apikey),time.sleep(1.2))
    assert sumreq.status_code == 200
    accids.append(sumreq.json()['accountId'])

In [12]:
# get game ids from match history via match-api
# 1.3 delay bc long list

gamids = []

for acc in accids:
    matlreq = requests.get("https://euw1.api.riotgames.com/lol/match/v4/matchlists/by-account/{}?api_key={}"
                           .format(acc,apikey),time.sleep(1.2))
    
    i=0
    while matlreq.status_code != 200:
        matlreq = requests.get("https://euw1.api.riotgames.com/lol/match/v4/matchlists/by-account/{}?api_key={}"
                           .format(acc,apikey),time.sleep(1.2))
        i+=1
        if i == 5:
            break
    
    
    gamids += [d['gameId'] for d in matlreq.json()['matches']]

In [14]:
matlreq.status_code

200

In [15]:
# print original length, then correct overlap from shared games and print new length

print(len(gamids))

gamids = set(gamids)

print(len(gamids))

16488
16488


In [16]:
with open('euw_chall_gam_id_list.txt', 'r') as f:
    challgames = f.read().splitlines()

In [17]:
challgames =set([int(e) for e in challgames])

In [18]:
with open('gm_gam_id_list.txt', 'r') as f:
    gmgames = f.read().splitlines()

In [19]:
gmgames =set([int(e) for e in gmgames])

In [20]:
gamids = gamids - (gamids & (gmgames | challgames))

In [21]:
len(gamids)

12663

In [23]:
with open('mas_gam_id_list.txt', 'w') as f:
    for item in gamids:
        f.write("%s\n" % item)

In [25]:
gamids = list(gamids)

In [72]:
# get individual match information from gamids list via match-api
# 1.5 delay bc long list and repeated issues with 

matchlist = []

for gam in gamids[12000:]:
    gamreq = requests.get("https://euw1.api.riotgames.com/lol/match/v4/matches/{}?api_key={}"
                          .format(gam,apikey),time.sleep(1.3))
    
    i=0
    while gamreq.status_code != 200:
        gamreq = requests.get("https://euw1.api.riotgames.com/lol/match/v4/matches/{}?api_key={}"
                          .format(gam,apikey),time.sleep(1.3))
        i+=1
        if i == 5:
            assert gamreq.status_code == 200
            
    match = flatten(gamreq.json())
    matchlist.append(match)

In [73]:
gamreq.status_code

200

In [74]:
# check length of matchlist 

check = [len(m) for m in matchlist]
len(matchlist),max(check),min(check)

(663, 1550, 1210)

In [75]:
# create df from matches

df = pd.DataFrame(matchlist)

In [76]:
df = df[df['gameMode'] == 'CLASSIC']

In [77]:
df

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,...,participants_4_timeline_damageTakenDiffPerMinDeltas_30-end,participants_5_timeline_csDiffPerMinDeltas_30-end,participants_5_timeline_xpDiffPerMinDeltas_30-end,participants_5_timeline_damageTakenDiffPerMinDeltas_30-end,participants_1_timeline_csDiffPerMinDeltas_30-end,participants_1_timeline_xpDiffPerMinDeltas_30-end,participants_1_timeline_damageTakenDiffPerMinDeltas_30-end,participants_8_timeline_csDiffPerMinDeltas_30-end,participants_8_timeline_xpDiffPerMinDeltas_30-end,participants_8_timeline_damageTakenDiffPerMinDeltas_30-end
1,4721341209,EUW1,1595492814448,1529,420,11,13,10.15.328.8759,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4726846233,EUW1,1595738161599,1396,420,11,13,10.15.328.8759,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4726256415,EUW1,1595707070610,927,420,11,13,10.15.328.8759,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4728615725,EUW1,1595805327959,1384,420,11,13,10.15.328.8759,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4715311920,EUW1,1595181354205,2195,420,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,4711186399,EUW1,1594931043990,1415,400,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
657,4669898721,EUW1,1592660587546,2092,420,11,13,10.12.325.6222,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
659,4670488558,EUW1,1592682190029,1127,420,11,13,10.12.325.6222,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
661,4720820220,EUW1,1595452118256,1938,420,11,13,10.15.328.8759,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
df1 = pd.read_csv("eumaster4.csv", low_memory=False)
df = df1.append(df)

In [42]:
df1 = pd.DataFrame(matchlist)
df1 = df1[df1['gameMode'] == 'CLASSIC']

In [43]:
df = df.append(df1)

In [44]:
df

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,...,participants_2_timeline_damageTakenDiffPerMinDeltas_30-end,participants_3_timeline_csDiffPerMinDeltas_30-end,participants_3_timeline_xpDiffPerMinDeltas_30-end,participants_3_timeline_damageTakenDiffPerMinDeltas_30-end,participants_6_timeline_csDiffPerMinDeltas_30-end,participants_6_timeline_xpDiffPerMinDeltas_30-end,participants_6_timeline_damageTakenDiffPerMinDeltas_30-end,participants_8_timeline_csDiffPerMinDeltas_30-end,participants_8_timeline_xpDiffPerMinDeltas_30-end,participants_8_timeline_damageTakenDiffPerMinDeltas_30-end
0,4734402147,EUW1,1596106410709,1653,420,11,13,10.15.330.344,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4678893160,EUW1,1593122691794,1925,420,11,13,10.13.325.7485,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4714806898,EUW1,1595162014831,1828,440,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4708056691,EUW1,1594762733170,1365,420,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4660805244,EUW1,1592173163312,1981,420,11,13,10.12.324.5925,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1264,4725242414,EUW1,1595654961520,2635,420,11,13,10.15.328.8759,CLASSIC,MATCHED_GAME,...,330.4,0.1,-193.75,330.4,NaN,NaN,NaN,1.0,108.3,-959.7
1267,4724390453,EUW1,1595621959126,1437,420,11,13,10.15.328.8759,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1268,4671437369,EUW1,1592748427154,936,420,11,13,10.12.325.6222,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1269,4706826814,EUW1,1594719536397,1908,420,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
df.to_csv('eumaster4.zip', index=False,
          compression=dict(method='zip',
                        archive_name='eumaster4.csv')  )

In [87]:
dfa = pd.read_csv("eumaster1.csv", low_memory=False)
dfb = pd.read_csv("eumaster2.csv", low_memory=False)
dfc = pd.read_csv("eumaster3.csv", low_memory=False)
dfd = pd.read_csv("eumaster4.csv", low_memory=False)

In [88]:
dfa.append([dfb,dfc,dfd])

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,...,participantIdentities_9_player_platformId,participantIdentities_9_player_accountId,participantIdentities_9_player_summonerName,participantIdentities_9_player_summonerId,participantIdentities_9_player_currentPlatformId,participantIdentities_9_player_currentAccountId,participantIdentities_9_player_matchHistoryUri,participantIdentities_9_player_profileIcon,teams_0_bans,teams_1_bans
0,4727570437,EUW1,1595783173528,2298,420,11,13,10.15.328.8759,CLASSIC,MATCHED_GAME,...,EUW1,5rFj62SGr7rEfSvMMqxJ4wZBy1StIpmjwwImvdQe0wzv-A,Monsieur Tioo,P8bRiA2H_Z_rH78tjQyJJy4dxBZJgcuBC_TQP1WDrHkbws4,EUW1,5rFj62SGr7rEfSvMMqxJ4wZBy1StIpmjwwImvdQe0wzv-A,/v1/stats/player_history/EUW1/22645148,4568,NaN,NaN
1,4719378438,EUW1,1595408311479,1443,420,11,13,10.15.328.8759,CLASSIC,MATCHED_GAME,...,EUW1,hoB-6VCqOEf1RZgIzCY5ZA8HbvKabZMr3RiU5qGJc-eMkYE,Fingornan,P5LjAhRtqFSu8OP2AghY2LLOtoqKNUbOfW3_Ag8BpqcdpEKi,EUW1,hoB-6VCqOEf1RZgIzCY5ZA8HbvKabZMr3RiU5qGJc-eMkYE,/v1/stats/player_history/EUW1/237239900,4271,NaN,NaN
2,4709875720,EUW1,1594884090749,1171,420,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,...,EUW1,E4XjRygu-kBFmsoo3JSnje4zG3jSNqt1Y4SqSR7F0uXknoM,Corne,MSfh-nuqYAHlkfzuL8qJDurhz0aMPU-jjQc0gulmVVL2bfc,EUW1,E4XjRygu-kBFmsoo3JSnje4zG3jSNqt1Y4SqSR7F0uXknoM,/v1/stats/player_history/EUW1/202138573,3014,NaN,NaN
3,4672389136,EUW1,1592775784419,1926,420,11,13,10.12.325.6222,CLASSIC,MATCHED_GAME,...,EUW1,-99-PR7YMlCJI7Vq9ArB2h48QrZAKRJw0SSjEFP984W_PqI,MTW InDeed,lM0g8IgbgkOTDm-btQRt5deiwEJOAVn53UPMjY2rEDxG1vE,EUW1,-99-PR7YMlCJI7Vq9ArB2h48QrZAKRJw0SSjEFP984W_PqI,/v1/stats/player_history/EUW1/204587938,4027,NaN,NaN
4,4715905041,EUW1,1595204969753,2141,420,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,...,EUW1,nq9fyYIGcI780lzpJxQkZo6jO41glxazVuL8rE1GaHaslgc,Snowhill,B15_AiekI8Fy8fWg9VsFXrqANaGYXU6KPjbPiykQDSPSoL2H,EUW1,nq9fyYIGcI780lzpJxQkZo6jO41glxazVuL8rE1GaHaslgc,/v1/stats/player_history/EUW1/238426611,4576,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3059,4711186399,EUW1,1594931043990,1415,400,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,...,EUW1,hsWaN634i7XfWWrnzbMS9yaBLiajlqAvyXSXJDMABdG594c,Rakhoon,l-9zDGQNz0Ta66MvBolhHOPqicLbGp629J32prHyqVMLllo,EUW1,hsWaN634i7XfWWrnzbMS9yaBLiajlqAvyXSXJDMABdG594c,/v1/stats/player_history/EUW1/225028212,3552,NaN,NaN
3060,4669898721,EUW1,1592660587546,2092,420,11,13,10.12.325.6222,CLASSIC,MATCHED_GAME,...,EUW1,1vvaq07aKqin9rbzqO8kBgoe8jv7dX90_UQeA0v0Aa_qfg,Symeοn,i07iBHA1a_vntjoT7opHCF5l1UG2vLFhoVuRmdEuuLPZEkM,EUW1,1vvaq07aKqin9rbzqO8kBgoe8jv7dX90_UQeA0v0Aa_qfg,/v1/stats/player_history/EUW1/29013077,589,NaN,NaN
3061,4670488558,EUW1,1592682190029,1127,420,11,13,10.12.325.6222,CLASSIC,MATCHED_GAME,...,EUW1,W0dniBSNVDUHvAd_w2I8ZkhHV7CxOjYvqsPNXR-t3DWfaA,IZI Nolan,Rgi1C_NpXqHCjetnC8Rr-mv5FYjNuL7xGvNjmp5D8oVWQfM,EUW1,W0dniBSNVDUHvAd_w2I8ZkhHV7CxOjYvqsPNXR-t3DWfaA,/v1/stats/player_history/EUW1/36152485,4568,NaN,NaN
3062,4720820220,EUW1,1595452118256,1938,420,11,13,10.15.328.8759,CLASSIC,MATCHED_GAME,...,EUW1,GqjXTQYwLjefQgdwjFbo5gDqSEKLz1YrioECVuB5GIhBz6...,RandomFjutTV,OHa9MN7Vn82wUOFiRqYazADmzAYiYiEOdLdyVsvpo6a6SM4P,EUW1,GqjXTQYwLjefQgdwjFbo5gDqSEKLz1YrioECVuB5GIhBz6...,/v1/stats/player_history/EUW1/2426790705891360,3798,NaN,NaN


In [89]:
dfa.to_csv('eumaster.zip', index=False,
          compression=dict(method='zip',
                        archive_name='eumaster.csv')  )

In [90]:
dfa = pd.read_csv("eugm1.csv", low_memory=False)
dfb = pd.read_csv("eugm2.csv", low_memory=False)
dfc = pd.read_csv("eugm3.csv", low_memory=False)

In [91]:
dfa.append([dfb,dfc])

,gameId,platformId,gameCreation,gameDuration,queueId,mapId,seasonId,gameVersion,gameMode,gameType,...,participants_8_timeline_damageTakenDiffPerMinDeltas_30-end,participants_9_timeline_creepsPerMinDeltas_30-end,participants_9_timeline_xpPerMinDeltas_30-end,participants_9_timeline_goldPerMinDeltas_30-end,participants_9_timeline_csDiffPerMinDeltas_30-end,participants_9_timeline_xpDiffPerMinDeltas_30-end,participants_9_timeline_damageTakenPerMinDeltas_30-end,participants_9_timeline_damageTakenDiffPerMinDeltas_30-end,teams_0_bans,teams_1_bans
0,4708499459,EUW1,1594799346359,1482,420,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4673011715,EUW1,1592823985273,1891,420,11,13,10.12.325.6222,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4674486280,EUW1,1592921264879,1511,420,11,13,10.12.325.6222,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4725506057,EUW1,1595675754704,2357,420,11,13,10.15.328.8759,CLASSIC,MATCHED_GAME,...,472.4,6.8,526.6,542.4,4.4,174.4,2987.8,1113.4,NaN,NaN
4,4733173777,EUW1,1596034276300,1097,440,11,13,10.15.330.344,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3909,4723605480,EUW1,1595594040069,838,1300,21,13,10.15.328.8759,NEXUSBLITZ,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3910,4696244211,EUW1,1594138740718,838,450,12,13,10.13.326.4870,ARAM,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
3911,4700602358,EUW1,1594359721851,1903,420,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3912,4709351417,EUW1,1594841284913,1147,420,11,13,10.14.327.8505,CLASSIC,MATCHED_GAME,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
dfa.to_csv('eugm.zip', index=False,
          compression=dict(method='zip',
                        archive_name='eugm.csv')  )